# Предствление текстовых документов в векторной форме

Пусть есть коллекция документов $D$. Будем рассматривать модель *bag-of-words* (мешок слов), то есть каждый документ состоит из какого-то набора слов (терма) без учета их позиций внутри документа. 

Рассмотрим коллекцию документов:

In [1]:
docs = ['человек лев орел черепаха человек', 
        'лев вол орел',
        'лев черепаха лев кошка',
        'жучка кошка мышка',
        'лев орел грифон']

Посчитаем сколько каждое слово встретилось во всей коллекции и сколько каждом документе:

In [2]:
import re
import numpy as np
import pandas as pd


from collections import defaultdict, Counter 

def parse_doc(doc):
    return re.split(r'\s+', doc, re.U)

# подсчет df по коллекции
def calc_df_dict(docs):
    c = Counter() 
    for doc_id, doc in enumerate(docs):
        c.update(set(parse_doc(doc)))
    return c 
    
# подсчет tf для документа
def calc_tf_dict(doc):
    c = Counter() 
    for word in parse_doc(doc):
        c[word] += 1
    return c 
    
dfs = calc_df_dict(docs)
pd.DataFrame(data=list(dfs.items()), columns=['term', 'df'])

,term,df
0,лев,4
1,черепаха,2
2,жучка,1
3,вол,1
4,кошка,2
5,орел,3
6,человек,1
7,мышка,1
8,грифон,1


Каждому слову и каждому документу можно присвоить уникальный численнй индефикатор, и построить так называемую tf-матрицу, состоящую из элементов $\{tf_{t,d}\}$ - вес слова $t$ в документе $d$ (где $t = 1 \dots n$ - индексы слов, $j = 1 \dots k$ - индексы документов). Под весом может подразумеваться число вхождний, нормализированная частота, и т.п.

### Система обозначений SMART

$\{tf_{t,d}\}$ не всегда дает адекватне предсавление (документы могут быть сильно не равной длины, не учитывается значимость слов). Поэтому существует более сложные модели под обобщеным названием *tf-idf* (term frequency - inverted document frequency), где используется документная частота слова. Идея в том, что для каждого слова в каждом документе считается *tf*, потом *idf* и значения перемножаются. Потом для каждого документа получается вектор, с которым можно что-то сделать (нормализовать).

<table width="100%">
<tr><th width="33%">Частота термина</th><th width="33%">Документная частота</th><th width="33%">Нормировка</th></tr>
<tr><td>n $$\text{tf}_{t,d}$$</td><td>n $$1$$</td><td>n $$1$$</td></tr>
<tr><td>l $$1 + \log{\text{tf}_{t,d}}$$</td> <td>t $$\log{\frac{N}{\text{df}_i}}$$</td><td>c $$\frac{1}{\sqrt{\omega_1^2 + \ldots + \omega_m^2}}$$</td></tr>

<tr><td>a $$0.5 + \frac{0.5 \text{tf}_{t,d}}{\max_t{ \text{tf}_{t,d}}}$$</td><td>p $$\text{max}(0, \log{\frac{N - \text{df}_i}{\text{df}_i}})$$</td><td></td></tr>

<tr><td>b $$  \begin{cases}
    1, & \text{if }  \text{tf}_{t,d} > 0 \\
    0, & \text{otherwise}
    \end{cases}$$
    </td><td></td></tr>

<tr><td>L $$\frac{1 + \log{\text{tf}_{t,d}}}{1 + \log{\text{avg}_{t \in d}(\text{tf}_{t,d})}}$$</td> <td></td></tr>

</table>

### Ипользование scikit-learn

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit_transform(docs).toarray()  

array([[0, 0, 0, 0, 1, 0, 1, 2, 1],
       [1, 0, 0, 0, 1, 0, 1, 0, 0],
       [0, 0, 0, 1, 2, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 1, 0, 1, 0, 0]], dtype=int64)

In [4]:
print('\n'.join(vectorizer.get_feature_names()))

вол
грифон
жучка
кошка
лев
мышка
орел
человек
черепаха
